Hi People! 

This is my first scraping.
Tools:
1. Jupyter Lab
2. Beautiful Soup

Reason:
I really like superhero movies, especially superhero landing *kidding ^_^. My favorite superhero is Batman, but I'm not a fanboy DC pleasee :(( I also like MCU, maybe because she likes Loki(?) can you guess what I like?
That's right, I love Moonknight, because he has a superhero landing, three crazy personalities, and he's a vigilante like Batman but without a personal code 
*WhiteBatman btw(*^^*)

So in this project, I scrap from IMDb with some criteria: 
1. Superhero 
2. At least 5000 Number of votes
3. Feature Film

If you guys have any question or opinion please do not hesitate DM me on Twitter ^_^

Link:
https://www.imdb.com/search/keyword/?keywords=superhero&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=a581b14c-5a82-4e29-9cf8-54f909ced9e1&pf_rd_r=N2C82Y6JW3V357NKJ6RV&pf_rd_s=center-5&pf_rd_t=15051&pf_rd_i=genre&ref_=kw_nxt&mode=detail&page=1&title_type=movie&num_votes=5000%2C&sort=num_votes,desc

In [1]:
import pandas as pd
import requests as rq
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs
import csv
from random import randint
import numpy as np

In [2]:
#user agent from my browser
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36' 


In [3]:
movie_name = []
year = []
time = []
genre = []
rating = []
metascore = []
director = []
votes = []
gross = []
pages = np.arange(1,5)

In [4]:
#Take 200 Movies
for req in pages:
    head = {"User-Agent": user_agent}
    req = rq.get(("https://www.imdb.com/search/keyword/?keywords=superhero&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=a581b14c-5a82-4e29-9cf8-54f909ced9e1&pf_rd_r=N2C82Y6JW3V357NKJ6RV&pf_rd_s=center-5&pf_rd_t=15051&pf_rd_i=genre&ref_=kw_nxt&mode=detail&page="+str(req)+"&title_type=movie&num_votes=5000%2C&sort=num_votes,desc"),headers=head)
    soup = bs(req.text, 'html.parser')
    row= soup.findAll('div',class_='lister-item mode-detail')
    #sleep(randint(2,8))
    #title movie
    for i in row:
        name = i.h3.a.text.strip()
        movie_name.append(name)
    
    #year or release    
    for i in row:
        release = i.h3.find('span','lister-item-year text-muted unbold').text.replace('(','').replace(')','').replace('I ','').replace('IX ','')
        year.append(release)
        
    #duration of each film   
    for i in row:    
        duration =i.p.find('span','runtime').text.replace('min','').replace(' ','')#.astype(int)
        time.append(duration)
        
    #genre
    for i in row:
        type_movie=i.p.find('span','genre').text.replace('\n','').replace(' ','')
        genre.append(type_movie)
        
    #rating of each film
    for i in row:
        rate=i.find('div','inline-block ratings-imdb-rating').text.replace('\n\n','').replace('\n','')
        rating.append(rate)
    
    #metascore
    for i in row:
        meta=i.find('div','inline-block ratings-metascore').text.replace(' ','').replace('\n','').replace('Metascore','') if i.find('div','inline-block ratings-metascore') else None
        metascore.append(meta)
    
    #number of vote and gross
    for i in row:
        vg=i.find_all('span',attrs = {'name':'nv'})
        #in this iteration, have 2 values for vote and gross
        vote=vg[0].text.replace(',','')
        gr=vg[1].text.replace('$','') if len(vg)>1 else None#.replace('M','') 
        votes.append(vote)
        gross.append(gr)
    
    #director of film
    for i in row:
        direc=i.find_all('p','text-muted text-small')[1].text.replace('\n    ','').replace('\n','').replace('Director:','').replace('Directors:','')
        direc=direc.split('|')[0]
        director.append(direc)

In [5]:
df = pd.DataFrame({'Title of Movies': movie_name,
                   'Year': year,
                   'Duration': time,
                   'Genre': genre,
                   'Rating': rating,
                   'Metascore': metascore,
                   'Director(s)': director,
                   'Votes':votes,
                   'Gross ($)': gross,
                  })

In [7]:
df

,Title of Movies,Year,Duration,Genre,Rating,Metascore,Director(s),Votes,Gross ($)
0,The Dark Knight,2008,152,"Action,Crime,Drama",9.0,84,Christopher Nolan,2539670,534.86M
1,The Dark Knight Rises,2012,164,"Action,Crime,Drama",8.4,78,Christopher Nolan,1643689,448.14M
2,Batman Begins,2005,140,"Action,Crime,Drama",8.2,70,Christopher Nolan,1418993,206.85M
3,The Avengers,2012,143,"Action,Adventure,Sci-Fi",8.0,69,Joss Whedon,1348778,623.28M
4,Guardians of the Galaxy,2014,121,"Action,Adventure,Comedy",8.0,76,James Gunn,1134189,333.18M
...,...,...,...,...,...,...,...,...,...
195,Inspector Gadget,1999,78,"Action,Adventure,Comedy",4.1,36,David Kellogg,47876,97.40M
196,Howard the Duck,1986,110,"Action,Adventure,Comedy",4.7,28,Willard Huyck,46540,16.30M
197,Spies in Disguise,2019,102,"Animation,Action,Adventure",6.8,54,"Nick Bruno, Troy Quane",46265,66.76M
198,A Nightmare on Elm Street: The Dream Child,1989,89,"Fantasy,Horror",5.0,54,Stephen Hopkins,44929,22.17M


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title of Movies  200 non-null    object
 1   Year             200 non-null    object
 2   Duration         200 non-null    object
 3   Genre            200 non-null    object
 4   Rating           200 non-null    object
 5   Metascore        197 non-null    object
 6   Director(s)      200 non-null    object
 7   Votes            200 non-null    object
 8   Gross ($)        190 non-null    object
dtypes: object(9)
memory usage: 14.2+ KB


In [6]:
#convert to csv and excel
df.to_csv('200 Movie Superhero IMDb.csv', index=False, encoding='utf-8')
df.to_excel('200 Movie Superhero IMDb.xlsx', index=False, encoding='utf-8')

If you all want RAW data in excel or csv, DM me on twitter. Because I will use this data for analysis, and maybe I need a little bit of your help for a better discussion of analyzing this data.
'cause, two heads are better than one isn't it?